In [165]:
# Importing
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from xgboost import XGBRegressor

pd.set_option("display.max_columns", None)


df = pd.read_csv(
    "https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv"
)

crime_rate = pd.read_csv("./crime_rate.csv")

# mean_income = pd.read_csv("./seattle_mean_yearly_gross_income.csv")

In [166]:
# Add crime rates from nearest city
df = pd.merge(df, crime_rate, on="zipcode", how="left")
# df = pd.merge(df, mean_income, on="zipcode", how="left")

In [167]:
def data_transform(df):
    df =df.drop('school_district', axis=1)

    # Convert 'date' to datetime and extract year and month
    df["date"] = pd.to_datetime(df["date"])

    df["year_of_sale"] = df["date"].dt.year
    df["month_of_sale"] = df["date"].dt.month

    df.drop("date", axis=1, inplace=True)  # Drop the original date column

    # Add age of house

    df["age"] = 2015 - df["yr_built"]

    # Add binary has basement column

    df["has_basement"] = (df["sqft_basement"] > 0).astype(int)

    CURRENT_YEAR = 2015  
    df["years_since_renovation"] = df["yr_renovated"].apply(
        lambda x: CURRENT_YEAR - x if x > 0 else 0
    )

    df.drop("yr_renovated", axis=1, inplace=True)

    

    return df

In [168]:
df = data_transform(df)

In [169]:
df.describe()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,zipcode,lat,long,sqft_living15,sqft_lot15,price,property_crime_rate_closest_city,violent_crime_rate_closest_city,school_rank,year_of_sale,month_of_sale,age,has_basement,years_since_renovation
count,2.000000e+04,20000.000000,20000.000000,20000.000000,2.000000e+04,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000,2.000000e+04,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000
mean,4.580297e+09,3.370200,2.115750,2079.883100,1.501533e+04,1.49645,0.007550,0.234900,3.409700,7.657700,1790.232900,289.650200,1971.100150,98077.876850,47.559970,-122.213897,1986.387500,12748.96040,5.394367e+05,3749.200515,238.292960,2.971450,2014.321300,6.586250,43.899850,0.391000,0.78960
std,2.875295e+09,0.931931,0.770134,919.433672,3.970135e+04,0.54095,0.086564,0.767692,0.651895,1.175287,829.587101,441.537072,29.374008,53.438026,0.138517,0.140649,685.211515,27076.48904,3.664334e+05,2595.934536,232.729265,1.776596,0.466987,3.114586,29.374008,0.487987,4.92293
min,1.000102e+06,0.000000,0.000000,290.000000,5.200000e+02,1.00000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,98001.000000,47.155900,-122.519000,399.000000,651.00000,7.500000e+04,621.700000,0.000000,1.000000,2014.000000,1.000000,0.000000,0.000000,0.00000
25%,2.123049e+09,3.000000,1.750000,1430.000000,5.040000e+03,1.00000,0.000000,0.000000,3.000000,7.000000,1200.000000,0.000000,1951.750000,98033.000000,47.470675,-122.328000,1490.000000,5100.00000,3.220000e+05,2203.200000,28.600000,1.000000,2014.000000,4.000000,18.000000,0.000000,0.00000
50%,3.904955e+09,3.000000,2.250000,1910.000000,7.620000e+03,1.50000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,98065.000000,47.571450,-122.230000,1840.000000,7622.00000,4.500000e+05,3091.500000,109.700000,3.000000,2014.000000,6.000000,40.000000,0.000000,0.00000
75%,7.309825e+09,4.000000,2.500000,2550.000000,1.071000e+04,2.00000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,98117.000000,47.678000,-122.124000,2360.000000,10088.25000,6.416250e+05,5435.500000,392.500000,4.000000,2015.000000,9.000000,63.250000,1.000000,0.00000
max,9.900000e+09,33.000000,8.000000,13540.000000,1.164794e+06,3.50000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.00000,7.700000e+06,16583.800000,750.600000,7.000000,2015.000000,12.000000,115.000000,1.000000,81.00000


In [170]:
# features = [
#     'id',
#     "bathrooms",
#     "bedrooms",
#     "sqft_living",
#     "floors",
#     "waterfront",
#     "view",
#     "grade",
#     "sqft_above",
#     "sqft_basement",
#     "lat",
#     "sqft_living15",
#     "school_rank",
#     "property_crime_rate_closest_city",
#     "violent_crime_rate_closest_city",
# ]
X = df.drop("price", axis=1)
y = df[["price"]]


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=10
)

In [171]:
model = XGBRegressor()


model.fit(X_train, y_train)

predictions = model.predict(X_test)


# Calculate RMSE
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)

r2 = r2_score(y_test, predictions)

print(f"\n\nRMSE: {rmse}")
print(f"\n\nR^2: {r2}")



RMSE: 119255.57685400377


R^2: 0.8946861933516949


In [172]:
mini = pd.read_csv(
    "https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test_mini.csv"
)


mini = pd.merge(mini, crime_rate, on="zipcode", how="left")


In [173]:
mini = data_transform(mini)

In [174]:
predictions = model.predict(mini)
df = pd.DataFrame(predictions)
df.to_csv("team3-module3-predictions.csv", index=False, header=["price"])

In [175]:
holdout = pd.read_csv(
    "https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test.csv"
)

holdout = pd.merge(holdout, crime_rate, on="zipcode", how="left")

In [176]:
holdout = data_transform(holdout)


In [177]:
# predictions = model.predict(holdout)
# df = pd.DataFrame(predictions)
# df.to_csv("team3-module3-predictions.csv", index=False, header=["price"])